# 电力负荷数据合成：解锁电网分析新可能

在现代电力系统中，数据是驱动洞察和优化的核心。然而，获取、管理和共享真实电力负荷数据常常面临隐私、成本和数据稀疏性等挑战。**电力负荷数据合成**提供了一个强大的解决方案，它利用先进的机器学习技术来生成与真实数据统计特性相似的**人造数据**。这不仅能有效弥补数据不足，还能在保护隐私的前提下，为电网的分析、预测、优化及算法测试提供丰富、多样化的数据支持。

### 为何合成电力负荷数据如此重要？

* **隐私保护与合规性：** 真实的电力数据往往包含敏感的用户行为信息。合成数据允许在不暴露原始数据的情况下进行分析和共享，满足严格的数据隐私法规要求。
* **弥补数据不足：** 面对传感器故障、通信中断或特定历史数据缺失时，合成数据能够填补空白，构建更完整、连续的数据集。
* **数据增强与多样化：** 通过生成不同条件（如极端天气、突发事件）下的数据，我们可以扩充现有数据集，提高模型的泛化能力和鲁棒性，让算法在更广泛的场景下表现出色。
* **算法测试与开发：** 无论是新的负荷预测模型还是需求响应策略，都需要大量数据进行训练和验证。合成数据提供了一个安全、可控的环境，加速算法的迭代和优化。
* **降低成本：** 相较于真实数据的采集、清洗和存储，合成数据显著降低了数据获取的成本和复杂度。



![research_overview](4ee8a7ae1a3c096a9b72318594b2b13.png)

---

### 核心合成模型与方法

电力负荷数据合成主要依赖于**深度生成模型**，这些模型擅长学习复杂数据的高维分布并生成逼真样本。

#### 1. 条件变分自编码器 (CVAE)

我们此次实现的核心模型就是 **CVAE**。

* **工作原理：** CVAE 能够学习在给定**条件信息**（例如：温度、湿度、风速、日期和时间等环境与时序特征）时，电力负荷数据（`PowerConsumption_Zone1, Zone2, Zone3`）的分布。它将输入数据编码成一个**潜在空间**中的概率分布，再由解码器结合条件信息从该潜在空间中采样并重建数据。
* **优势：** CVAE 能够生成具有多样性的数据，并且最关键的是，它允许我们**控制生成过程**，根据特定的环境和时间条件来定制合成的电力负荷数据，这对于模拟特定场景下的电力行为至关重要。

#### 2. 生成对抗网络 (GANs)

* **工作原理：** GANs 由一个**生成器**和一个**判别器**组成。生成器负责创造看起来真实的合成数据，而判别器则努力区分真实数据和生成数据。两者在对抗中不断提升，最终生成器能够产生高度逼真的数据。
* **优势：** 在生成视觉上极度逼真的数据方面表现卓越，也能应用于时序数据。

#### 3. 扩散模型 (Diffusion Models)

* **工作原理：** 这类模型通过模拟一个逐步添加噪声的“前向扩散”过程，然后学习逆向的“去噪”过程来生成数据。
* **优势：** 近年来在图像和音频生成领域取得了突破性进展，能够生成高质量、细节丰富的样本，在复杂数据分布建模方面潜力巨大，是未来在电力数据合成领域值得探索的方向。



![what_we_do](cvae_architecture_diagram.svg)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# --- 环境设置 ---
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['font.sans-serif'] = ['Misans']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# --- 读取和预处理数据 ---
df = pd.read_csv("./data/powerconsumption.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.sort_values('Datetime').reset_index(drop=True)

# 提取时间特征 (作为类别特征)
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Month'] = df['Datetime'].dt.month - 1

# 定义连续条件特征、类别条件特征和目标特征
continuous_conditional_features = ['Temperature', 'Humidity', 'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows']
categorical_conditional_features = ['Hour', 'DayOfWeek', 'Month']
target_features = ['PowerConsumption_Zone1', 'PowerConsumption_Zone2', 'PowerConsumption_Zone3']

# 划分训练集和测试集 (80/20)
train_size = int(len(df) * 0.8)
train_df, test_df = df.iloc[:train_size], df.iloc[train_size:]

# 对连续特征和目标特征进行归一化
scaler_cond_continuous = MinMaxScaler()
scaler_target = MinMaxScaler()

# 训练集缩放
train_df[continuous_conditional_features] = scaler_cond_continuous.fit_transform(
    train_df[continuous_conditional_features])
train_df[target_features] = scaler_target.fit_transform(train_df[target_features])

# 测试集缩放
test_df[continuous_conditional_features] = scaler_cond_continuous.transform(test_df[continuous_conditional_features])
test_df[target_features] = scaler_target.transform(test_df[target_features])

# 准备 Tensors
# 训练数据
X_train_cont = torch.tensor(train_df[continuous_conditional_features].values, dtype=torch.float32)
X_train_cat = torch.tensor(train_df[categorical_conditional_features].values, dtype=torch.long)
y_train = torch.tensor(train_df[target_features].values, dtype=torch.float32)

# 测试数据
X_test_cont = torch.tensor(test_df[continuous_conditional_features].values, dtype=torch.float32)
X_test_cat = torch.tensor(test_df[categorical_conditional_features].values, dtype=torch.long)
y_test = torch.tensor(test_df[target_features].values, dtype=torch.float32)

# 创建 DataLoader
batch_size = 256
train_dataset = TensorDataset(X_train_cont, X_train_cat, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_cont, X_test_cat, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("--- 数据维度检查 ---")
print(f"训练集-连续条件: {X_train_cont.shape}")
print(f"训练集-类别条件: {X_train_cat.shape}")
print(f"训练集-目标: {y_train.shape}")
print(f"测试集-连续条件: {X_test_cont.shape}")
print(f"测试集-类别条件: {X_test_cat.shape}")
print(f"测试集-目标: {y_test.shape}")

使用设备: cuda
--- 数据维度检查 ---
训练集-连续条件: torch.Size([41932, 5])
训练集-类别条件: torch.Size([41932, 3])
训练集-目标: torch.Size([41932, 3])
测试集-连续条件: torch.Size([10484, 5])
测试集-类别条件: torch.Size([10484, 3])
测试集-目标: torch.Size([10484, 3])


![the_priciple_of_generation](34488b69a3202d7dc6a3631b30d3a6e.png)

In [3]:
class UpgradedCVAE(nn.Module):
    def __init__(self, target_dim, cont_cond_dim, latent_dim,
                 cat_dims, embedding_dims):
        super(UpgradedCVAE, self).__init__()

        self.target_dim = target_dim
        self.latent_dim = latent_dim

        # 1. 条件特征嵌入层
        self.hour_embed = nn.Embedding(cat_dims['Hour'], embedding_dims['Hour'])
        self.day_embed = nn.Embedding(cat_dims['DayOfWeek'], embedding_dims['DayOfWeek'])
        self.month_embed = nn.Embedding(cat_dims['Month'], embedding_dims['Month'])

        total_embedding_dim = embedding_dims['Hour'] + embedding_dims['DayOfWeek'] + embedding_dims['Month']
        conditional_dim = cont_cond_dim + total_embedding_dim

        # 2. 编码器 (Encoder)
        encoder_input_dim = target_dim + conditional_dim
        self.encoder_net = nn.Sequential(
            nn.Linear(encoder_input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_log_var = nn.Linear(128, latent_dim)

        # 3. 解码器 (Decoder) - 多任务结构
        decoder_input_dim = latent_dim + conditional_dim
        self.decoder_shared = nn.Sequential(
            nn.Linear(decoder_input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU()
        )
        # 为每个Zone创建独立的输出头
        self.decoder_zone1 = nn.Linear(256, 1)
        self.decoder_zone2 = nn.Linear(256, 1)
        self.decoder_zone3 = nn.Linear(256, 1)

    def process_conditions(self, c_cont, c_cat):
        # c_cat shape: (batch_size, num_cat_features)
        hour_emb = self.hour_embed(c_cat[:, 0])
        day_emb = self.day_embed(c_cat[:, 1])
        month_emb = self.month_embed(c_cat[:, 2])

        c_emb = torch.cat([hour_emb, day_emb, month_emb], dim=1)
        c_combined = torch.cat([c_cont, c_emb], dim=1)
        return c_combined

    def encode(self, x, c_cont, c_cat):
        c = self.process_conditions(c_cont, c_cat)
        inputs = torch.cat([x, c], dim=1)
        h = self.encoder_net(inputs)
        return self.fc_mu(h), self.fc_log_var(h)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c_cont, c_cat):
        c = self.process_conditions(c_cont, c_cat)
        inputs = torch.cat([z, c], dim=1)

        h_shared = self.decoder_shared(inputs)

        out1 = self.decoder_zone1(h_shared)
        out2 = self.decoder_zone2(h_shared)
        out3 = self.decoder_zone3(h_shared)

        # 将三个Zone的输出拼接起来，并用Sigmoid确保范围在[0,1]
        return torch.sigmoid(torch.cat([out1, out2, out3], dim=1))

    def forward(self, x, c_cont, c_cat):
        mu, log_var = self.encode(x, c_cont, c_cat)
        z = self.reparameterize(mu, log_var)
        reconstruction = self.decode(z, c_cont, c_cat)
        return reconstruction, mu, log_var

    @torch.no_grad()
    def generate(self, c_cont, c_cat):
        z = torch.randn(c_cont.size(0), self.latent_dim, device=c_cont.device)
        return self.decode(z, c_cont, c_cat)

In [4]:
def mmd_loss(p, q, kernel='rbf', sigma=1.0):
    """计算p和q两个batch之间的MMD损失"""
    # p: 真实样本 (batch_size, dim)
    # q: 生成样本 (batch_size, dim)
    xx, yy, zz = torch.mm(p, p.t()), torch.mm(q, q.t()), torch.mm(p, q.t())

    # RBF 核 (高斯核)
    rx = (xx.diag().unsqueeze(0).expand_as(xx))
    ry = (yy.diag().unsqueeze(0).expand_as(yy))

    Kxx = torch.exp(-0.5 * (rx.t() + rx - 2 * xx) / sigma ** 2)
    Kyy = torch.exp(-0.5 * (ry.t() + ry - 2 * yy) / sigma ** 2)
    Kxy = torch.exp(-0.5 * (rx.t() + ry - 2 * zz) / sigma ** 2)

    return Kxx.mean() + Kyy.mean() - 2 * Kxy.mean()


def combined_loss_function(recon_x, x, mu, log_var, generated_x,
                           zone_weights, beta=1.0, alpha=1.0):
    """
    计算总损失 = 加权重建损失 + beta * KLD + alpha * MMD
    """
    # 1. 加权重建损失 (Weighted MSE)
    recon_loss_per_zone = torch.mean((recon_x - x) ** 2, dim=0)  # 按Zone计算MSE
    weighted_recon_loss = torch.sum(recon_loss_per_zone * zone_weights)

    # 2. KL 散度损失
    kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    kld_loss /= x.size(0)  # 平均到每个样本

    # 3. MMD 损失
    mmd = mmd_loss(x, generated_x)

    # 4. 组合损失
    total_loss = weighted_recon_loss + beta * kld_loss + alpha * mmd

    return total_loss, weighted_recon_loss, kld_loss, mmd

In [5]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from tqdm.notebook import tqdm
# ... your existing code ...

In [ ]:
# --- 模型超参数 ---
TARGET_DIM = y_train.shape[1]
CONT_COND_DIM = X_train_cont.shape[1]
LATENT_DIM = 30
CAT_DIMS = {'Hour': 24, 'DayOfWeek': 7, 'Month': 12}
EMBEDDING_DIMS = {'Hour': 8, 'DayOfWeek': 4, 'Month': 4}

# # --- 训练超参数:old ---
# EPOCHS = 100
# LEARNING_RATE = 1e-4
# BETA = 0.1  # KLD损失的权重 (可调)
# ALPHA = 10.0  # MMD损失的权重 (可调)
# # 为Zone3赋予更高权重
# ZONE_WEIGHTS = torch.tensor([1.0, 1.0, 3.0]).to(device)

# --- 训练超参数 ---
EPOCHS = 50
LEARNING_RATE = 1e-4
BETA = 0.05  # KLD损失的权重 (可调)
ALPHA = 10.0  # MMD损失的权重 (可调)
# 为Zone3赋予更高权重
ZONE_WEIGHTS = torch.tensor([1.0, 1.0, 0.5]).to(device)

# --- 初始化 ---
model = UpgradedCVAE(
    target_dim=TARGET_DIM,
    cont_cond_dim=CONT_COND_DIM,
    latent_dim=LATENT_DIM,
    cat_dims=CAT_DIMS,
    embedding_dims=EMBEDDING_DIMS
).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- 训练与评估循环 ---
train_losses, test_mses = [], []

for epoch in tqdm(range(EPOCHS)):
    model.train()
    total_loss, total_recon, total_kld, total_mmd = 0, 0, 0, 0

    # 使用tqdm进行训练可视化
    pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{EPOCHS}", unit="batch")

    for c_cont, c_cat, x in pbar:
        c_cont, c_cat, x = c_cont.to(device), c_cat.to(device), x.to(device)

        optimizer.zero_grad()

        # 前向传播
        recon_x, mu, log_var = model(x, c_cont, c_cat)

        # 为MMD损失生成样本
        generated_x = model.generate(c_cont, c_cat)

        # 计算损失
        loss, recon, kld, mmd = combined_loss_function(
            recon_x, x, mu, log_var, generated_x, ZONE_WEIGHTS, BETA, ALPHA
        )

        # 反向传播和优化
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_recon += recon.item()
        total_kld += kld.item()
        total_mmd += mmd.item()

        pbar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Recon': f'{recon.item():.4f}',
            'KLD': f'{kld.item():.4f}',
            'MMD': f'{mmd.item():.4f}'
        })

    avg_loss = total_loss / len(train_loader)
    avg_recon = total_recon / len(train_loader)
    avg_kld = total_kld / len(train_loader)
    avg_mmd = total_mmd / len(train_loader)
    print(
        f"Epoch {epoch + 1} Avg Train Loss: {avg_loss:.4f} | Recon: {avg_recon:.4f} | KLD: {avg_kld:.4f} | MMD: {avg_mmd:.4f}")

    # --- 在测试集上评估 ---
    model.eval()
    all_recon_x = []
    all_x = []
    with torch.no_grad():
        for c_cont, c_cat, x in test_loader:
            c_cont, c_cat, x = c_cont.to(device), c_cat.to(device), x.to(device)
            recon_x, _, _ = model(x, c_cont, c_cat)
            all_recon_x.append(recon_x)
            all_x.append(x)

    all_recon_x = torch.cat(all_recon_x, dim=0)
    all_x = torch.cat(all_x, dim=0)

    # 计算每个Zone的MSE
    test_mse = torch.mean((all_recon_x - all_x) ** 2, dim=0)
    test_mses.append(test_mse.cpu().numpy())

    print(f"Epoch {epoch + 1} Test MSE -> "
          f"Zone1: {test_mse[0]:.6f}, "
          f"Zone2: {test_mse[1]:.6f}, "
          f"Zone3: {test_mse[2]:.6f} (Weighted)")
    print("-" * 80)

print("训练完成!")



In [ ]:
# --- 结果可视化 ---
test_mses_np = np.array(test_mses)
plt.figure(figsize=(12, 6))
plt.plot(test_mses_np[:, 0], label='Zone 1 MSE')
plt.plot(test_mses_np[:, 1], label='Zone 2 MSE')
plt.plot(test_mses_np[:, 2], label='Zone 3 MSE (Weighted)', linestyle='--')
plt.title('各区域在测试集上的MSE随Epoch变化')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# --- 预测和真实对比可视化 ---
model.eval()
all_recon_x_unscaled = []
all_x_unscaled = []
all_c_cont_test = []
all_c_cat_test = []

with torch.no_grad():
    for c_cont, c_cat, x in test_loader:
        c_cont, c_cat, x = c_cont.to(device), c_cat.to(device), x.to(device)
        recon_x, _, _ = model(x, c_cont, c_cat)

        # 将预测和真实值从GPU移回CPU并存储
        all_recon_x_unscaled.append(recon_x.cpu().numpy())
        all_x_unscaled.append(x.cpu().numpy())
        all_c_cont_test.append(c_cont.cpu().numpy())
        all_c_cat_test.append(c_cat.cpu().numpy())

# 将所有批次的预测和真实值拼接起来
all_recon_x_unscaled = np.concatenate(all_recon_x_unscaled, axis=0)
all_x_unscaled = np.concatenate(all_x_unscaled, axis=0)

# 使用 MinMax scaler 反转缩放
all_recon_x_unscaled = scaler_target.inverse_transform(all_recon_x_unscaled)
all_x_unscaled = scaler_target.inverse_transform(all_x_unscaled)

# 选择前1000个点进行可视化
num_points_to_plot = 24 * 6 * 10

plt.figure(figsize=(18, 12))

for i, zone in enumerate(target_features):
    plt.subplot(len(target_features), 1, i + 1)
    plt.plot(all_x_unscaled[:num_points_to_plot, i], label=f'实际 {zone}', color='blue', alpha=0.7)
    plt.plot(all_recon_x_unscaled[:num_points_to_plot, i], label=f'预测 {zone}', color='red', linestyle='--', alpha=0.7)
    plt.title(f'测试集前{num_points_to_plot}个点：{zone} 预测 vs 实际')
    plt.xlabel('时间点')
    plt.ylabel('电量消耗')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# --- 预测和真实对比可视化与置信区间 ---
model.eval()

# 获取测试集数据
# We will use the first batch or a selected subset of the test set for visualization
# For simplicity, let's take the first 'num_points_to_plot' data points directly from tensors
# Make sure these are on the correct device
X_test_cont_plot = X_test_cont[:num_points_to_plot].to(device)
X_test_cat_plot = X_test_cat[:num_points_to_plot].to(device)
y_test_plot = y_test[:num_points_to_plot].to(device)

num_samples_for_ci = 1000  # Number of samples to generate for each test point to build the CI

# Store multiple predictions for each test point
# Shape: (num_points_to_plot, num_samples_for_ci, target_dim)
all_generated_samples_scaled = torch.zeros(
    num_points_to_plot, num_samples_for_ci, TARGET_DIM
).to(device)

print(f"Generating {num_samples_for_ci} samples for each of the first {num_points_to_plot} test points...")

with torch.no_grad():
    for s in tqdm(range(num_samples_for_ci), desc="Sampling for CI"):
        # For each sample, we pass the same conditional inputs to the model's generate method
        # This will produce different outputs due to the reparameterization trick / sampling from Z
        generated_s_scaled = model.generate(X_test_cont_plot, X_test_cat_plot)
        all_generated_samples_scaled[:, s, :] = generated_s_scaled

# Move to CPU and convert to NumPy
all_generated_samples_scaled_np = all_generated_samples_scaled.cpu().numpy()
y_test_plot_np = y_test_plot.cpu().numpy()

# Inverse transform everything to original scale
# (num_points_to_plot, num_samples_for_ci, target_dim)
all_generated_samples_unscaled = np.array([
    scaler_target.inverse_transform(all_generated_samples_scaled_np[i, :, :])
    for i in range(num_points_to_plot)
])

# Inverse transform the actual test values
y_test_plot_unscaled = scaler_target.inverse_transform(y_test_plot_np)

# Calculate the mean prediction and confidence intervals (e.g., 95%)
mean_predictions = np.mean(all_generated_samples_unscaled, axis=1)  # Shape: (num_points_to_plot, target_dim)
lower_bound = np.percentile(all_generated_samples_unscaled, 2.5, axis=1)  # 2.5th percentile
upper_bound = np.percentile(all_generated_samples_unscaled, 97.5, axis=1)  # 97.5th percentile

# --- 可视化 ---
num_points_to_plot_vis = 24 * 7 * 5  # Let's plot 2 weeks for clearer visualization of CI

plt.figure(figsize=(18, 12))

for i, zone in enumerate(target_features):
    plt.subplot(len(target_features), 1, i + 1)
    plt.plot(y_test_plot_unscaled[:num_points_to_plot_vis, i], label=f'实际 {zone}', color='blue', alpha=0.8)
    plt.plot(mean_predictions[:num_points_to_plot_vis, i], label=f'预测均值 {zone}', color='red', linestyle='-',
             alpha=0.8)
    plt.fill_between(
        range(num_points_to_plot_vis),
        lower_bound[:num_points_to_plot_vis, i],
        upper_bound[:num_points_to_plot_vis, i],
        color='red', alpha=0.2, label='95% 置信区间'
    )
    plt.title(f'测试集前{num_points_to_plot_vis}个点：{zone} 预测均值与95%置信区间 vs 实际')
    plt.xlabel('时间点')
    plt.ylabel('电量消耗')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# plt.figure(figsize=(15, 4))
# for i, zone in enumerate(target_features):
#     plt.subplot(1, len(target_features), i + 1)
#     sns.kdeplot(all_x_unscaled[:, i], label=f'实际 {zone}', color='blue', fill=True, alpha=0.6)
#     sns.kdeplot(all_recon_x_unscaled[:, i], label=f'预测 {zone}', color='red', linestyle='--', fill=True, alpha=0.4)
#     plt.title(f'{zone} 预测 vs 实际分布')
#     plt.xlabel('电量消耗')
#     plt.ylabel('密度')
#     plt.legend()
#     plt.grid(True)
# 
# plt.tight_layout()
# plt.show()

In [ ]:
def generate_power_consumption(input_data):
    """
    根据提供的环境和时间条件，生成 Zone 1、2 和 3 的合成电力消耗数据。

    参数:
        input_data (pd.Series 或 dict): 包含输入条件的一行数据，
                                        键名需与原始数据列名匹配：
                                        'Datetime', 'Temperature', 'Humidity',
                                        'WindSpeed', 'GeneralDiffuseFlows', 'DiffuseFlows'。

    返回:
        dict: 包含生成的 'PowerConsumption_Zone1'、
              'PowerConsumption_Zone2' 和 'PowerConsumption_Zone3' 的字典。
    """
    # 1. 将 input_data 转换为 DataFrame 以进行统一处理
    if isinstance(input_data, dict):
        input_df = pd.DataFrame([input_data])
    elif isinstance(input_data, pd.Series):
        input_df = pd.DataFrame([input_data.to_dict()])
    else:
        raise ValueError("input_data 必须是 pandas Series 或 dictionary 类型。")

    # 确保 'Datetime' 列是日期时间格式
    input_df['Datetime'] = pd.to_datetime(input_df['Datetime'])

    # 2. 提取时间特征 (与训练数据预处理保持一致)
    input_df['Hour'] = input_df['Datetime'].dt.hour
    input_df['DayOfWeek'] = input_df['Datetime'].dt.dayofweek
    input_df['Month'] = input_df['Datetime'].dt.month  # Keep as 1-12 for encoding

    # 3. 准备连续条件特征和类别条件特征 (不进行独热编码，而是直接送入模型的 Embedding 层)
    # 提取连续条件特征
    c_cont_input = input_df[continuous_conditional_features].values.astype(np.float32)

    # 提取类别条件特征
    # IMPORTANT: These should be the raw integer categorical values,
    # as your UpgradedCVAE model takes `c_cat` as a `torch.long` tensor
    # where each column is an index for an embedding.
    # If your original training adjusted month to 0-11, this needs to be consistent here.
    # Assuming `Month` is 1-12 from input_df and `CAT_DIMS` handles it.
    # If `Month` was `month-1` in training, then it should be `input_df['Month'] - 1` here.
    c_cat_input = input_df[categorical_conditional_features].values.astype(np.int64)

    # 4. 使用预先拟合的 scaler_cond_continuous 对连续输入特征进行归一化
    c_cont_scaled = scaler_cond_continuous.transform(c_cont_input)

    # 将 NumPy 数组转换为 PyTorch 张量并移动到指定设备 (CPU/GPU)
    c_cont_tensor = torch.tensor(c_cont_scaled, dtype=torch.float32).to(device)
    c_cat_tensor = torch.tensor(c_cat_input, dtype=torch.long).to(device)

    # 5. 使用训练好的 CVAE 模型生成合成数据
    model.eval()  # 将模型设置为评估模式 (不计算梯度，不进行 dropout 等)
    with torch.no_grad():  # 禁用梯度计算，节省内存并加速推理
        # 生成样本，CVAE的generate方法只需要条件特征
        generated_y_scaled = model.generate(c_cont_tensor, c_cat_tensor).cpu().numpy()

    # 6. 将生成的归一化数据反归一化回原始尺度
    generated_y_unscaled = scaler_target.inverse_transform(generated_y_scaled)

    # 7. 格式化输出为字典
    output_dict = {
        'PowerConsumption_Zone1': generated_y_unscaled[0, 0],
        'PowerConsumption_Zone2': generated_y_unscaled[0, 1],
        'PowerConsumption_Zone3': generated_y_unscaled[0, 2]
    }
    return output_dict


# --- 示例用法 ---
# 确保您的 'model'、'scaler_cond_continuous'、'scaler_target'、'LATENT_DIM'、'device'
# 这些变量已经通过之前的训练步骤准备好并可用。
# 上面我已经为这些变量提供了模拟的定义，以便此代码块可以直接运行。

print("--- 运行生成函数示例 ---")

# 示例输入数据 1 (字典形式)
example_input_data = {
    'Datetime': '2017-01-01 00:00:00',
    'Temperature': 6.559,
    'Humidity': 73.8,
    'WindSpeed': 0.083,
    'GeneralDiffuseFlows': 0.051,
    'DiffuseFlows': 0.119
}

generated_power = generate_power_consumption(example_input_data)
print("\n生成的电力消耗 (示例 1 - 新年午夜):")
for zone, value in generated_power.items():
    print(f"  {zone}: {value:.2f}")

# 示例输入数据 2 (夏季下午，潜在用电高峰)
example_input_data_2 = {
    'Datetime': '2017-07-15 14:30:00',  # 夏季下午
    'Temperature': 35.0,
    'Humidity': 60.0,
    'WindSpeed': 2.5,
    'GeneralDiffuseFlows': 800.0,
    'DiffuseFlows': 250.0
}
generated_power_2 = generate_power_consumption(example_input_data_2)
print("\n生成的电力消耗 (示例 2 - 夏季下午):")
for zone, value in generated_power_2.items():
    print(f"  {zone}: {value:.2f}")

# 示例输入数据 3 (寒冷夜晚)
example_input_data_3 = {
    'Datetime': '2017-01-20 03:00:00',  # 冬季夜晚
    'Temperature': 5.0,
    'Humidity': 85.0,
    'WindSpeed': 0.1,
    'GeneralDiffuseFlows': 0.01,
    'DiffuseFlows': 0.01
}
generated_power_3 = generate_power_consumption(example_input_data_3)
print("\n生成的电力消耗 (示例 3 - 寒冷夜晚):")
for zone, value in generated_power_3.items():
    print(f"  {zone}: {value:.2f}")


In [ ]:
# 示例输入数据 (Any data from df)
example_input_data_3 = df.iloc[1, :6].to_dict()
generated_power_3 = generate_power_consumption(example_input_data_3)
print("\n生成的电力消耗 (寒冷夜晚):")
for zone, value in generated_power_3.items():
    print(f"  {zone}: {value:.2f}")

df.iloc[1, 6:-3]  # ground truth

In [ ]:
# --- 生成前2000行数据的预测并添加到DataFrame ---

# 确保 df 已经加载并且是原始未缩放的数据
# (如果您之前修改了df，可能需要重新加载或确保它包含原始值)
# 例如: df = pd.read_csv("./data/powerconsumption.csv")
# df['Datetime'] = pd.to_datetime(df['Datetime'])

# 确定要处理的行数
num_rows_to_predict = 5000

# 准备存储预测结果的列表
predictions_zone1 = []
predictions_zone2 = []
predictions_zone3 = []

# 定义输入 generate_power_consumption 函数所需的列
# 'Datetime' 必须包含在输入中以提取时间特征
input_cols_for_generation = [
    'Datetime', 'Temperature', 'Humidity', 'WindSpeed',
    'GeneralDiffuseFlows', 'DiffuseFlows'
]

print(f"正在为 df 的前 {num_rows_to_predict} 行生成预测...")

# 遍历 df 的前 num_rows_to_predict 行
# 使用 tqdm 包装，提供进度条
for i in tqdm(range(num_rows_to_predict), desc="生成预测"):
    # 从当前行提取输入数据，转换为字典
    current_input_data = df.iloc[i][input_cols_for_generation].to_dict()

    # 使用 generate_power_consumption 函数获取预测
    generated_power = generate_power_consumption(current_input_data)

    # 将预测结果添加到相应的列表中
    predictions_zone1.append(generated_power['PowerConsumption_Zone1'])
    predictions_zone2.append(generated_power['PowerConsumption_Zone2'])
    predictions_zone3.append(generated_power['PowerConsumption_Zone3'])

# 将预测结果作为新列添加到 df
# 注意：这些新列将只包含前 num_rows_to_predict 行的数据，其余为 NaN
df.loc[:num_rows_to_predict - 1, 'PowerConsumption_Zone1_Predict'] = predictions_zone1
df.loc[:num_rows_to_predict - 1, 'PowerConsumption_Zone2_Predict'] = predictions_zone2
df.loc[:num_rows_to_predict - 1, 'PowerConsumption_Zone3_Predict'] = predictions_zone3

print(f"成功将预测结果添加到 df 中，并创建了新的预测列。")

# 打印更新后的 df 的头部和新的预测列信息，以供检查
print("\n更新后的 df (前5行) 及新预测列:")
print(df[['Datetime'] + target_features + [col for col in df.columns if 'Predict' in col]].head())

# 可以选择进一步打印一些特定行的对比
print(f"\n第1行（索引为0）的实际与预测对比:")
print(
    f"实际 Zone1: {df.loc[0, 'PowerConsumption_Zone1']:.2f}, 预测 Zone1: {df.loc[0, 'PowerConsumption_Zone1_Predict']:.2f}")
print(
    f"实际 Zone2: {df.loc[0, 'PowerConsumption_Zone2']:.2f}, 预测 Zone2: {df.loc[0, 'PowerConsumption_Zone2_Predict']:.2f}")
print(
    f"实际 Zone3: {df.loc[0, 'PowerConsumption_Zone3']:.2f}, 预测 Zone3: {df.loc[0, 'PowerConsumption_Zone3_Predict']:.2f}")

print(f"\n第1000行（索引为999）的实际与预测对比:")
print(
    f"实际 Zone1: {df.loc[999, 'PowerConsumption_Zone1']:.2f}, 预测 Zone1: {df.loc[999, 'PowerConsumption_Zone1_Predict']:.2f}")
print(
    f"实际 Zone2: {df.loc[999, 'PowerConsumption_Zone2']:.2f}, 预测 Zone2: {df.loc[999, 'PowerConsumption_Zone2_Predict']:.2f}")
print(
    f"实际 Zone3: {df.loc[999, 'PowerConsumption_Zone3']:.2f}, 预测 Zone3: {df.loc[999, 'PowerConsumption_Zone3_Predict']:.2f}")

In [ ]:
# df[['Datetime', 'PowerConsumption_Zone1',
#     'PowerConsumption_Zone2', 'PowerConsumption_Zone3', 'PowerConsumption_Zone1_Predict',
#     'PowerConsumption_Zone2_Predict', 'PowerConsumption_Zone3_Predict']].iloc[:2000][::10].plot(x='Datetime',
#                                                                                                 figsize=(18, 5))

In [ ]:
import matplotlib.pyplot as plt

# 采样（每隔10个取1个，前2000个）
df_sample = df.iloc[:2000]

# 创建子图
fig, axs = plt.subplots(3, 1, figsize=(18, 12), sharex=True)

# 区域1
axs[0].plot(df_sample['Datetime'], df_sample['PowerConsumption_Zone1'], label='Zone1 Real')
axs[0].plot(df_sample['Datetime'], df_sample['PowerConsumption_Zone1_Predict'], label='Zone1 Predict')
axs[0].set_title('Zone 1 Power Consumption')
axs[0].legend()

# 区域2
axs[1].plot(df_sample['Datetime'], df_sample['PowerConsumption_Zone2'], label='Zone2 Real')
axs[1].plot(df_sample['Datetime'], df_sample['PowerConsumption_Zone2_Predict'], label='Zone2 Predict')
axs[1].set_title('Zone 2 Power Consumption')
axs[1].legend()

# 区域3
axs[2].plot(df_sample['Datetime'], df_sample['PowerConsumption_Zone3'], label='Zone3 Real')
axs[2].plot(df_sample['Datetime'], df_sample['PowerConsumption_Zone3_Predict'], label='Zone3 Predict')
axs[2].set_title('Zone 3 Power Consumption')
axs[2].legend()

# 总体设置
plt.xlabel('Datetime')
plt.tight_layout()
plt.show()

In [ ]:
!jupyter nbconvert --to html energy_CVAE4.ipynb 
# !jupyter nbconvert --to markdown energy_CVAE2.ipynb